# 8 客户关系管理
在客户关系管理这一概念成为时髦术语之前，许多组织设计并开发了以客户为中心的维度模型，以便能够更好地理解客户的行为。客户关系管理是一种重要的商业策略，很多观念认为，客户关系管理是企业生存的基础  
本章首先综述客户关系管理的基本概念，包含其操作和分析角色。在描述各种不同的客户层次时，将探讨建模复杂层次存在的问题  
本章主要包含以下概念：  
+ 客户关系管理概述
+ 客户名称和地址解析，包括国际方面的考虑 5  
+ 处理日期、聚集事实，以及在客户维度中分割行为属性和积分  
+ 用于处理低粒度属性的支架表  
+ 稀疏数据的桥接表，以及权衡桥接表与位置设计  
+ 应用于多客户联系人的桥接表
+ 获取客户队列组的行为研究分组 5
+ 分析有序客户行为的步骤维度  
+ 带有有效日期和失效日期的包含时间范围的事实表
+ 完善事实表使用的满意维度或异常情况  
+ 在ETL过程下游通过主数据管理或部分一致性集成客户数据  
+ 有关对应表连接的警告 5  
+ 实现实时、低时间延迟需求的检查  

因为本章以客户为中心的建模问题和模式与跨行业和功能区的情况是相关的，因此不讨论总线矩阵 5  
## 8.1 客户关系管理概述
无论哪个行业，组织结构都蜂拥而至推崇客户关系管理这一概念。客户关系管理必须关注业务的各个方面，包括市场、销售、服务、操作等方面，建立并维持与客户之间的共同利益关系  
客户关系管理能够为接受它的企业带来明显的利益，不仅带来收入的增加，而且能够提高运作效率  
在许多组织中，对客户的理解根据产品链的业务单元、业务功能、地理位置的不同而存在差异  
过去几十年来，社会媒介、位置跟踪技术、网络使用监控、多媒体应用、传感器网络的爆炸性增长提供了海量的客户行为数据，即使街道企业也认识到适用它们可获得可操作的知识 5  
拒绝改变是人类的天性，毫不奇怪的是，与人有关的问题通常成为客户关系管理实现的难题  
在第17章中，将强调为建立DW/BI系统获得业务高层与IT管理部门支持的重要性  
### 操作型与分析型客户关系管理
可以说客户关系管理患有人格分裂症，因为它需要同时满足操作型和分析型需求  
从操作角度来看，客户关系管理需要同步客户面对的过程。不要将这些过程当成是独立的系统，客户关系管理的观念是将这些客户活动集成  
当数据在客户关系管理等价的操作端被建立后，显然需要存储和分析来自客户交互和交易系统的历史度量。数据仓库是支持客户全方位全景视图的基础 5  
分析型客户关系管理可确保获得DW/BI系统中精确的、集成的、可访问的客户数据。模型的结果也可以保留在DW/BI环境中以方便后续的分析  
![8-1](../syn_pic/the_datawarehouse_toolkit/8-1.png)
<center>图8-1 闭环分析型客户关系管理</center>

在其他环境中，信息必须以一种更实时的基础来反馈到操作型网站或呼叫中心。在此情况下，闭环比图8-1所示的情况有更紧密的方式，因为这种方式要收集并存储，然后反馈到集成系统中  
显然，组织越来越关注客户，也越来越需要DW/BI系统。最重要的是，一致性客户维度的需求变得更加关键 5  
## 8.2 客户维度属性
一致性客户维度是建立高效客户关系管理的关键元素  
客户维度通常是所有DW/BI系统维度中最具挑战性的维度。更为复杂的情况是，客户维度通常标识的是融合了多个内部和外部源系统的集成数据  
下一节，我们将关注多种客户维度设计考虑。从客户处获得的描述性信息越多，客户维度就越健壮，分析就更有意思  
### 8.2.1 名字与地址的语法分析
无论您处理的是个体的人还是商业实体，通常都需要获取客户姓名和地址属性。考察图8-2所示的用一般方法建立的列  
![8-2](../syn_pic/the_datawarehouse_toolkit/8-2.png)
<center>图8-2 过于一般化的客户姓名/地址示例数据 5</center>

这样设计的姓名/地址数据列将会受到太多限制。很难采用一致的机制处理称谓、标题和前缀  
在示例的地址属性中，不同地方采用的缩写形式不一  
与其使用通用意义的列，不如将姓名和地址属性拆分为多个可能的部分。抽取过程需要针对原先混乱的姓名和地址进行语法分析  
图8-3展示的是美国人地址和姓名属性。为方便理解，每个属性都包含一个示例数据，但真实的示例与展现的不同  
![8-3](../syn_pic/the_datawarehouse_toolkit/8-3.png)
<center>图8-3 包含姓名和地址语法分析的示例客户姓名/地址数据 5</center>

商业客户可能包含多个地址，例如，实体工厂地址和商店地址，每个地址都应该遵守图8-3展示的地址结构的逻辑含义  
### 8.2.2 国际姓名和地址的考虑
国际化展示和打印通常需要表示外文字符，不仅包括来自西欧的重音字符，也包括斯拉夫语、阿拉伯语、日语和中文，以及其他一些并不为人所熟悉的书写系统  
国际系统构建组织Unicode协会定义了一个称为Unicode的标准，用于表示世界上几乎所有国家的语言和文化所涉及的字符和字母  
需要注意的是，实现Unicode解决方案位于系统的基础层  
除了操作系统，所有用于获取、存储、转换和打印字符的设备都必须支持Unicode 5  
如果要处理的客户来自多个国家，客户地理属性会变得相当复杂  
注意：有时客户维度可能包括完整的地址块属性。该属性可用于那些有当地特色的国际位置  
**国际DW/BI目标**  
讨论了Unicode基础后，除了前面讨论的名称和地址分析需求以外，还需要牢记以下目标：  
+ **通用型和一致性**。如果打算使设计系统能够适合国际环境，让它能够在世界各地工作，则需要仔细考虑，BI工具是否产生多种语言的报表转换版本  
    + 排序序列存在差异，因此要么报表按照不同方式排序，要么除了那些采用“基本”语言的报表，其他所有报表都讲以无序方式出现 5 
    + 如果属性粒度在跨月宴环境下未能忠实保留，则要么分组总计将会出现差异，要么不同的语言的某些分组将包含看起来不对的重复行表头  
    + 所有BI工具的消息和提示符需要转换以方便用户使用  
+ **端到端数据质量以及下游的兼容性**  
    在整个数据流程中，数据仓库不是唯一需要考虑国际化姓名和地址的地方  
    从数据清洗和存储步骤开始，到最后一步执行地理和人口统计分析及打印报表步骤的整个过程中，都需要提供设计方面的考虑，以实现对获取名称和地址的支持  
+ **文化的正确性**  
    多数情况下，国外客户和合作伙伴将以某种方式获取DW/BI系统的最终结果。如果您不知道哪个名称的哪个部分是姓，哪个部分是名，不知道如何称呼人，那么您将会冒不尊重他人的风险，或者至少，看起来非常愚蠢 5  
+ **实时客户响应**  
    DW/BI系统可以通过支持实时客户响应系统，扮演一个操作型角色。客户服务代理可以回答电话，或者在不多于5秒左右的等待后从屏幕上得到一个数据仓库推荐使用的问候  
+ **其他类型的地址**  
    我们正置身与一场通信与网络的革命中。如果设计的系统能够处理国际姓名和地址，则必须预选考虑处理电子姓名、安全标志和网络地址  
    
### 8.2.3 客户为中心的日期
客户维度通常包含多种日期，例如，首次购买日期、最近一次购买的日期、生日等等  
尽管这些日期最初可能是SQL日期类型的列，但如果希望根据特定的日历属性汇总这些日期（例如，按照季节、季度、财务周期等等），则这些日期必须转变为引用日期维度的外键  
如图8-4所示的设计一种维度支架表，将在8.2.7小节讨论 5  
![8-4](../syn_pic/the_datawarehouse_toolkit/8-4.png)
<center>图8-4 日期维度支架表</center>

### 8.2.4 作为维度属性的聚集事实
商业用户通常喜欢基于聚集的性能度量约束客户维度，例如，从所有用户中过滤出那些在上一年度中花费超过一定数额的客户。将聚集事实作为维度属性提供给用户，将会使客户满意  
虽然存储这些属性可获得查询的可用性和改进性能，但主要的负担都落到ETL过程中，ETL过程需要确保属性的精确性，确保它们是最新的，并与实际的事实表行保持一致  
与其将属性存储为美元值，不如将它们替换为描述值，例如下一节将谈论的高额购买者。这些描述值减少了无法将数字属性与适当事实表联系的脆弱性  
此外，要保证所有用户对高额购买者有一致的定义，例如。不要使用自己定义的业务规则 5  
### 8.2.5 分段属性与记分
客户维度中最强有力的属性是分段类。对某个个体来说，肯能包括:  
+ 性别
+ 民族
+ 年龄或其他生命分段方式
+ 收入或其他生活类型分类 5
+ 状态（例如，新客户、活跃客户、不活跃客户、已离去客户）  
+ 参考源
+ 特定业务市场分段（例如，优先客户标识符）

类似地，许多组织为客户打分以刻画客户情况。每个客户用所得的分数标记  
1. 行为标记时间序列  
    一个常用的客户评分及分析系统方法是考察客户行为的相关度(R)、频繁度（F)和强度（I)。该方法被称为RFI方法 5  
    在处理大型客户数据库时，某个客户的行为可以按照如图8-5所示的RFI多维数据库建模  
    ![8-5](../syn_pic/the_datawarehouse_toolkit/8-5.png)
    <center>图8-5 相关度、频繁度和强度（RFI)多维数据库</center>

    如果在多维数据库中有上百万个点，则要理解不同分组的含义就比较困难。数据挖掘专家可能会利用下列的行为标识，更复杂一点的场景可能会包含信用行为和回报情况：  
    1. A高容量常客户，信誉良好，产品回报一般
    2. B高容量常客户，信誉良好，产品回报多 5
    3. C最近的新用户，尚未建立信誉模式
    4. D偶尔出现的客户，信誉良好
    5. E偶尔出现的客户，信誉不好
    6. F以前的优秀客户，最近不常见
    7. G只逛不买的客户，几乎没有效益
    8. H其他客户 5  
    
   至此可以考察客户时间序列数据并将某个客户关联到报表期间的最近分类中。这样，在最近10个考察期间，名为约翰的客户情况可以表示为：  
   约翰：C C C D D A A A B B  
   这一行为时间序列标记是不寻常的，因为它来自于固定周期度量过程，观察值是文本类型的。行为标记不是数字型的，不能计算或求平均值，但是它们可以被查询  
   行为标记可能不会被当成普通事实存储。行为标记的主要作用在于为类似前一段描述的例子制定复杂的查询模式。如果行为标记被存储在不同的事实表行中，此类查询将非常困难，需要串联关联的子查询  
   推荐的处理行为标记的方法是在客户维度中建立属性的时间序列。这也是位置设计的一个例子 5  
   除了为每个行为标记时间周期建立不同的列，建立单一的包含将所有行为标记连在一起的属性也是非常好的一种方法，例如，CCCDDAAABB。该列支持通配符搜索外部模式，例如，“D后紧跟着B”  
   注意：除了客户维度时间序列行为标记外，在微型维度中包含一个当前行为标示值也是非常合理的  
2. 数据挖掘与DW/BI之间的关系  
    数据挖掘小组是数据仓库的重要客户，是客户行为数据的特别重要的用户。然而，数据仓库用户发布数据的速度与数据挖掘用户使用数据的速度存在不匹配的情况  
    数据挖掘小组可能会喜欢这样的数据！例如，包含上百万此类查询结果的大型文件可以用决策树工具分析，该工具定位于Total Purchases Lifetime列，如上加粗文字  
    但是数据挖掘小组希望反复使用此类查询结果用于不同种类的分析工具，可能使用人工神经网络或者基于示例的推理工具 5  
    与其反复为数据仓库小组建立复杂的查询来生产庞大、昂贵的查询结果，不如将查询结果集合写入一个文件中，让数据挖掘小组在他们自己的服务器中分析  
    
### 8.2.6 包含类型2维度变化的计算
在业务上通常希望基于客户的属性，而不必与事实表连接来实现对客户的计算  